In [1]:
! pip install pinecone-client
! pip install faunadb
! pip install sentence_transformers
! pip install transformers

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import json
from sentence_transformers import SentenceTransformer, util
from langchain.chat_models import ChatOpenAI
from transformers import AutoTokenizer

In [ ]:
#Max tokensize per embedding = 384
model = SentenceTransformer('sentence-transformers/msmarco-distilbert-cos-v5')
tokenizer = AutoTokenizer.from_pretrained('pritamdeka/S-PubMedBert-MS-MARCO')

In [ ]:
import pinecone

pinecone.init(api_key="cbe1a9d3-f28d-465f-b224-2027dad6c00f", environment="gcp-starter")
index_1 = pinecone.Index("findata-fixed-chunk-size")

pinecone.init(api_key="0fde16e6-29bf-4ed6-846b-06cbbb93739e", environment="gcp-starter")
index_2 = pinecone.Index("findataparagraphsizedchunks")

In [ ]:
#Initialize FaunaDB
from faunadb import query as q
from faunadb.objects import Ref
from faunadb.client import FaunaClient

client_text = FaunaClient(
  secret="fnAFVT8S4NAAzBacYgW0ftLTvP6dWhGhceDR_lbV",
)

client_list = FaunaClient(
  secret="fnAFVT8ugqAAzfkg0VQscJmeOabEGwvzXyIpeKCH",
)

In [ ]:
def return_document_text(query):
  embeded_vector = model.encode(query).tolist()

  query_response = index_1.query(
      embeded_vector,
      top_k=3,
      )
  data = get_k_answers_text(query_response)
  return data

def get_k_answers_text(answers):
    query1 = answers["matches"][0]["id"]
    query2 = answers["matches"][1]["id"]
    query3 = answers["matches"][2]["id"]
    data1 = client_text.query(
        q.paginate(q.match(q.index("metadata"), query1))
    )
    data2 = client_text.query(
        q.paginate(q.match(q.index("metadata"), query2))
    )
    data3 = client_text.query(
        q.paginate(q.match(q.index("metadata"), query3))
    )
    text1 = data1["data"][0][1] + data1["data"][0][0] + data1["data"][0][2]
    text2 = data2["data"][0][1] + data2["data"][0][0] + data2["data"][0][2]
    text3 = data3["data"][0][1] + data3["data"][0][0] + data3["data"][0][2]
    print(text1)
    return text1

def return_document_list(query):
  embeded_vector = model.encode(query).tolist()

  query_response = index_2.query(
      embeded_vector,
      top_k=3,
      )
  data =get_k_answers_list(query_response)
  return data

def get_k_answers_list(answers):
    query1 = answers["matches"][0]["id"]
    query2 = answers["matches"][1]["id"]
    query3 = answers["matches"][2]["id"]
    data1 = client_list.query(
        q.paginate(q.match(q.index("metadata"), query1))
    )
    data2 = client_list.query(
        q.paginate(q.match(q.index("metadata"), query2))
    )
    data3 = client_list.query(
        q.paginate(q.match(q.index("metadata"), query3))
    )
    data = data1["data"][0][0] + "\n" + data2["data"][0][0] + "\n" + data3["data"][0][0]
    print(data)
    return data
    


In [ ]:
s = "What is the responsibility of Walmart Inc.'s management in relation to internal control over financial reporting?"

context_list = return_document_list(s)
print("\n \n")
context_text = return_document_text(s)

In [ ]:
from langchain.schema import SystemMessage,HumanMessage,AIMessage

query = "Can you tell me something about the context?"

augmented_prompt = f"""using the context below, answer the query

Context:
{context_list}

Query: {query}"""



message = [SystemMessage(content= "You are an helpfull assistant that has to answer questions based on some additional context. If the answer to a question does not appear in the text say so."), HumanMessage(content=  augmented_prompt)]

print(augmented_prompt)


In [ ]:
res = chat(message)